In [1]:
import numpy as np
import scipy as sp
from scipy import sparse
from scipy import integrate
from scipy.sparse import linalg
import matplotlib.pyplot as plt
from matplotlib import animation, rc
import time

#%matplotlib ipympl

plt.rcParams['figure.dpi']= 300
plt.rcParams['axes.facecolor']='white'
plt.rcParams['savefig.facecolor']='white'

In [11]:
## Problem 1 ##
L = 10
h = 0.1
xspan = np.linspace(-L, L, 201)[:-1]
tspan = np.arange(0,10.5,.5)
u_0 = np.exp(-1*(xspan - 5)**2)

## a ##
A = sp.sparse.spdiags([-1*np.ones(len(xspan)), np.ones(len(xspan))],
                      [-1,1], len(xspan), len(xspan), format = 'csc')/(2*h)
A[-1,0] = 1/(2*h)
A[0,-1] = -1/(2*h)
A1 = A.todense()

## b ##
def advection(t,x,A,c):
    deriv = -c*(A@x)
    return deriv

c = -0.5
sol = sp.integrate.solve_ivp(advection, [0,10], args = (A,c), y0 = u_0, 
                             t_eval = tspan, method = 'RK45')
A2 = sol.y

## c ##
def heaviside_advection(t,x,A,xspan):
    c = -(1 + 2*np.sin(5*t) - np.heaviside(xspan-4,0))
    x = -c*(A@x)
    return x

sol = sp.integrate.solve_ivp(heaviside_advection, [0,10], args = (A, xspan), 
                             y0 = u_0, t_eval = tspan, method = 'RK45')
A3 = sol.y

c:\Users\Bwa\anaconda3\lib\site-packages\scipy\sparse\_index.py:82: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


In [12]:
print(np.shape(A))

(200, 200)


In [6]:
A1[-1,0]

5.0

In [ ]:
## Problem 1 Presentation ##
## b ##
xspan = np.linspace(-L, L, 201)[:-1]
tspan = np.arange(0,10.5,.01)
X, T = np.meshgrid(xspan,tspan)

plt.subplot(211)
plt.imshow(-0.5*np.ones((21,200)), vmin = -0.75, vmax = 0.75, extent = (-L,L,0,10), 
           cmap = 'magma', aspect = 'auto', interpolation = 'antialiased')
plt.title('Temporal-Spatial evolution of constant and sinusoidal $c(x,t)$')
plt.ylabel('t')
plt.xticks([])
plt.subplot(212)
plt.imshow(-(1 + 2*np.sin(5*T) - np.heaviside(X-4,0)), vmin = -0.75, vmax = 0.75,  
           extent = (-L,L,0,10), cmap = 'magma', aspect = 'auto', interpolation = 'antialiased')
plt.title('Sinusoidal')
plt.xlabel('x')
plt.ylabel('t')
plt.subplots_adjust(bottom=0.1, right=0.8, top=0.9)
cax = plt.axes([0.85, 0.1, 0.075, 0.8])
plt.colorbar(cax=cax)
plt.show()

In [ ]:
## c ##
X, T = np.meshgrid(xspan,sol.t)
fig, ax = plt.subplots(1,2, subplot_kw={"projection": "3d"}, figsize = (6,3))
surf1 = ax[0].plot_surface(X, T, A2.T,cmap='magma')
surf2 = ax[1].plot_surface(X, T, A3.T,cmap='magma')
ax[0].set_xlabel('$x$')
ax[0].set_title('$c(x,t) = -0.5$', fontsize = 8)
ax[1].set_xlabel('$x$')
ax[1].set_ylabel('$t$')
ax[1].set_title('$c(x,t) = -(1+2\sin 5t - H(x-4))$', fontsize = 8)
ax[1].set_zlabel('$u(x,t)$')
fig.suptitle('Numerical Solution to Advection equation with varying $c(x,t)$')
plt.show()

In [3]:
## Problem 2 ##
nu = 0.001
L = 10
m = 64
n = m**2
dt = 0.5
t_max = 4

xspan = np.linspace(-L,L,m+1)[:-1]
yspan = np.linspace(-L,L,m+1)[:-1]
tspan = np.arange(0,t_max + dt,dt)

h = xspan[1] - xspan[0]

X, Y = np.meshgrid(xspan, yspan)
w_0 = np.exp(-2*X.T**2 - (Y.T**2)/20).flatten()

## a ##
e1 = np.ones(n) # vector of ones
Low1 = np.tile(np.concatenate((np.ones(m-1), [0])), (m,)) # Lower diagonal 1
Low2 = np.tile(np.concatenate(([1], np.zeros(m-1))), (m,)) #Lower diagonal 2

Up1 = np.roll(Low1, 1) # Shift the array for spdiags
Up2 = np.roll(Low2, m-1) # Shift the other array

A = sp.sparse.spdiags([e1, e1, Low2, Low1, -4*e1, Up1, Up2, e1, e1],
            [-(n-m), -m, -m+1, -1, 0, 1, m-1, m, (n-m)], n, n, format = 'csc')/h**2
A[0,0] = 2/h**2

B = sp.sparse.spdiags([e1, -e1, e1, -e1], [-(n - m), -m, m, (n-m)], 
                      n, n, format = 'csc')/(2*h)

C = sp.sparse.spdiags([Low2, -Low1, Up1, -Up2], [-m+1,-1,1,m-1], n, n, format = 'csc')/(2*h)

A4 = A.todense()
A5 = B.todense()
A6 = C.todense()

## bi ##
def gauss_vort(t, x, A, B, C, nu):
    psi = sp.sparse.linalg.spsolve(A,x)
    x = -((B@psi)*(C@x) - (C@psi)*(B@x)) + nu*(A@x)
    return x

t_gauss = time.time()
sol = sp.integrate.solve_ivp(gauss_vort, [0,t_max], y0 = w_0, args = (A,B,C,nu), 
                            t_eval = tspan, method = 'RK45')
t_gauss = time.time() - t_gauss

A7 = sol.y.T

## bii ##
lusolver = sp.sparse.linalg.splu(A)

def plu_vort(t, x, lusolver, B, C, nu):
    psi = lusolver.solve(x)
    x = -((B@psi)*(C@x) - (C@psi)*(B@x)) + nu*(A@x)
    return x

t_plu= time.time()
sol = sp.integrate.solve_ivp(plu_vort, [0,t_max], y0 = w_0, args = (lusolver,B,C,nu), 
                            t_eval = tspan, method = 'RK45')
t_plu = time.time() - t_plu

A8 = sol.y.T

A9 = sol.y.T.reshape(len(tspan),m,m)

In [ ]:
## Problem 2 Presentation (a) ##
nu = 0.001
L = 10
m = 64
n = m**2
dt = 0.1
t_max = 4

xspan = np.linspace(-L,L,m+1)[:-1]
yspan = np.linspace(-L,L,m+1)[:-1]
tspan = np.linspace(0,t_max, int(t_max/dt)+1)

h = xspan[1] - xspan[0]

X, Y = np.meshgrid(xspan, yspan)
w_0 = np.exp(-2*X.T**2 - (Y.T**2)/20).flatten()

lusolver = sp.sparse.linalg.splu(A)

def plu_vort(t, x, lusolver, B, C, nu):
    psi = lusolver.solve(x)
    x = -((B@psi)*(C@x) - (C@psi)*(B@x)) + nu*(A@x)
    return x

t_plu= time.time()
sol = sp.integrate.solve_ivp(plu_vort, [0,t_max], y0 = w_0, args = (lusolver,B,C,nu), 
                            t_eval = tspan, method = 'RK45')
t_plu = time.time() - t_plu

to_animate = sol.y.T.reshape(len(tspan),m,m)

In [ ]:
fig, ax = plt.subplots(1,1)
im = ax.pcolor(X,Y,to_animate[0,:,:].T, cmap = 'magma')
fig.colorbar(im)
ax.set_xlim([-9,9])
ax.set_ylim([-9,9])
ax.set_ylabel('y')
ax.set_xlabel('x')

def update(frame):
    im.set_array(to_animate[frame,:,:].T.ravel())
    ax.set_title(f"Vorticity ($\omega$) Evolution at time t = {np.round(tspan[frame],decimals = 1)}", loc = 'left', font = 'monospace')
    ax.set_xlim([-9,9])
    ax.set_ylim([-9,9])
    return im,

ani = animation.FuncAnimation(fig, update, frames = range(len(tspan)-1), interval = 1, blit = True)

writergif = animation.PillowWriter(fps = 24)
ani.save('vortex_true.gif', writer = writergif)